<a href="https://colab.research.google.com/github/moey920/NLP/blob/master/Document_Similarity_Jaccard_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문서 유사도(Document Similarity)


## 자카드 유사도

먼저 결론부터 요약하겠습니다.
자카드 유사도 : 두 문장을 각각 단어의 집합으로 만들고나서 -> 두 집합의 교집합/두 집합의 합집합
- [형태소 분리하고나서 진행했을 때 : 0.07% 향상]
 *  24659469.txt 와 24659469_hand.txt  비교시 0.691 유사
 *  24659469_masked.txt 와 24659469_hand.txt  비교시 0.698 유사


---

- [형태소 분리하지 않고 텍스트를 문자열로 받아와 비교했을 때 : ]

 문자열 단위
 *  24659469.txt 와 24659469_hand.txt  비교시 0.961 유사
 *  24659469_masked.txt 와 24659469_hand.txt  비교시 0.961 유사(변화없음)

 토큰화(단어) 단위
 *  24659469.txt 와 24659469_hand.txt  비교시 0.339 유사
 *  24659469_masked.txt 와 24659469_hand.txt  비교시 0.335 유사(0.04% 감소)


- 단점 : 각 문자, 단어 등의 위치를 고려하지 않고 집합으로 처리하는 알고리즘이라 영수증 유사도 비교에 큰 의미가 없는 것 같습니다

A와 B 두개의 집합이 있다고 합시다. 이때 교집합은 두 개의 집합에서 공통으로 가지고 있는 원소들의 집합을 말합니다. 즉, 합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있다는 것이 자카드 유사도(jaccard similarity)의 아이디어입니다. 자카드 유사도는 0과 1사이의 값을 가지며, 만약 두 집합이 동일하다면 1의 값을 가지고, 두 집합의 공통 원소가 없다면 0의 값을 갖습니다. 자카드 유사도를 구하는 함수를 J라고 하였을 때, 자카드 유사도 함수 J는 아래와 같습니다.

J(A,B)=|A∩B||A∪B|=|A∩B||A|+|B|−|A∩B|

두 개의 비교할 문서를 각각 doc1, doc2라고 했을 때 doc1과 doc2의 문서의 유사도를 구하기 위한 자카드 유사도는 이와 같습니다.

J(doc1,doc2)=doc1∩doc2doc1∪doc2

즉, 두 문서 doc1, doc2 사이의 자카드 유사도 J(doc1,doc2)는 두 집합의 교집합 크기를 두 집합의 합집합 크기로 나눈 값으로 정의됩니다. 간단한 예를 통해서 이해해보겠습니다.

### 예문

In [0]:
# 다음과 같은 두 개의 문서가 있습니다.
# 두 문서 모두에서 등장한 단어는 apple과 banana 2개.
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화를 수행합니다.
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

# 토큰화 결과 출력
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


이 때 문서1과 문서2의 합집합을 구해보겠습니다.

In [0]:
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)

{'like', 'holder', 'you', 'watch', 'card', 'passport', 'banana', 'coupon', 'love', 'everyone', 'likey', 'apple'}


문서1과 문서2의 합집합의 단어의 총 개수는 12개인 것을 확인할 수 있습니다. 그렇다면, 문서1과 문서2의 교집합을 구해보겠습니다. 즉, 문서1과 문서2에서 둘 다 등장한 단어를 구하게 됩니다.

In [0]:
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'banana', 'apple'}


문서1과 문서2에서 둘 다 등장한 단어는 banana와 apple 총 2개입니다. 이제 교집합의 수를 합집합의 수로 나누면 자카드 유사도가 계산됩니다.

In [0]:
print(len(intersection)/len(union)) # 2를 12로 나눔.

0.16666666666666666


위의 값은 자카드 유사도이자, 두 문서의 총 단어 집합에서 두 문서에서 공통적으로 등장한 단어의 비율이기도 합니다.

### 영수증 유사도 검출(형태소 분리하여 검사할 때)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import os

#user_file_link = '/content/drive/My Drive/문서 유사도/24659469.txt'
user_file_link = '/content/drive/My Drive/문서 유사도/24659469_masked.txt'
ground_file_link = '/content/drive/My Drive/문서 유사도/24659469_hand.txt'

train_data = pd.read_csv(user_file_link, header = 0, delimiter = '\t')
test_data = pd.read_csv(ground_file_link, header = 0, delimiter = '\t')

In [202]:
train_data.head(10)

,rio emart24
0,이마트24 영종대교 게소점
1,"대표자 : 김윤아, 성열 기"
2,사업자번호 : 478 85 -00613
3,"인천시 서구 정서진남로 25, 영종대교 휴게"
4,103145 2020 - 04 - 02 13 : 09 POS -0001 471
5,상품명 단가 수량 금액
6,01) 광동 ) 대추상화 150m /
7,"8806002011967 1, 500 1 1, 500"
8,02) e) 빅맨요구르트 280ml
9,"8801121025130 1, 200 1 1, 200"


In [203]:
test_data.head(10)

,emart24
0,이마트24 영종대교휴게소점
1,"대표자 : 김운아,성열기"
2,사업자번호 : 478-85-00613
3,"인천시 서구 정서진남로 25, 영종대교 휴게"
4,03145 2020-04-02 13:09 POS-0001 .47
5,----------------------------------------------...
6,상품명 단가 수량 금액
7,----------------------------------------------...
8,01) 광동)대추쌍화150ml
9,"8806002011967 1,500 1 1,500"


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 2.4MB 33.9MB/s 
     |████████████████████████████████| 92kB 11.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [0]:
import re
import json
from konlpy.tag import Okt

from tqdm import tqdm

In [0]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거. + 영어 소문자, 대문자, 숫자도 제외
    # 일단 OCR 결과의 원형을 학습시키기 위해 정규표현식을 사용하지 않고 학습시켜보겠습니다.
    #review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\\s]", " ",  review)
    #review_text = re.sub(" ", "",  review)
    review_text = review
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)
    
    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words]
        
   
    return word_review

In [206]:
#stop_words = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', 
#              '주', '등', '한', '(', ')', '/', '*', '=', 'E', '|', '-', '.', ',', 'II', 'لالالالا', 
#              '|||||||||', 'iii', '|||', '. ', '.', '"', ' )', '[', ']', '"']

stop_words = []

okt = Okt()
clean_train_review = []

for review in tqdm(train_data['rio emart24']):
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, remove_stopwords = False , stop_words=stop_words))
    else:
        clean_train_review.append([])  #string이 아니면 비어있는 값 추가

100%|██████████| 34/34 [00:00<00:00, 281.31it/s]


In [207]:
clean_test_review = []

for review in tqdm(test_data['emart24']):
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_test_review.append(preprocessing(review, okt, remove_stopwords = False , stop_words=stop_words))
    else:
        clean_test_review.append([])  #string이 아니면 비어있는 값 추가

100%|██████████| 38/38 [00:00<00:00, 607.71it/s]


In [0]:
sentences = clean_train_review
sentences2 = clean_test_review

리스트 내 리스트의 원소까지 모두 문자열로 flatten 하기 위한 코드

In [0]:
import itertools

def from_iterable(iterables):
    # chain.from_iterable(['ABC', 'DEF']) --> ['A', 'B', 'C', 'D', 'E', 'F']
    for it in iterables:
        for element in it:
            yield element

In [0]:
user = sentences
user = list(itertools.chain.from_iterable(user))

In [0]:
ground = sentences2
ground = list(itertools.chain.from_iterable(ground))

In [0]:
s1 = ' '.join(user)
s2 = ' '.join(ground)

In [0]:
s1 = s1.split()

In [0]:
s2 = s2.split()

In [215]:
union = set(s1).union(set(s2))
print(union)

{'인천', '85', '30일', '점', '롯데', '13', '차다', '★', '승인', '세', '휴게소', '시', '하다', '919490', '1,200', '과세', '앱', ':', '*]', '04', '열기', '22255237', '담당자', '사명', '대표자', '대', '환불', '금액', '정부', 'm', '00', '가다', '합', '2020-04', '이내', '드', '280', '남다', '**-****-', '휴게', '09', '가액', '을', '변심', '에서', '-', '부', '쌍', '500', '8801121025130', '.', '단가', '광동', 'e', '(', '화', '---------------------------------------------------', '월', '478', '요구르트', '가맹', '검색', '상품', '2,455', '번호', '700', '1,500', '소', '서구', '47', '명', '**-**', '으로', 'POS', '8806002011967', ')', '사업자', '13:09', '시불', '이마트', '1020200402031450001919490', '게', '성열', '추상화', '245', '에', '의하다', '02', '150', '455', '기다', '가능하다', '가져가다', '받다', '/', '단순', '계액', '영수증', '수량', '카드', '25', '6916-1500', '"', '020200402031450001919490', '식품', '보다', '빠르다', '김윤아', '2020', '결제', '103145', '33', '200', '03145', '계', '02-6916', '신용카드', '01', '영종대교', '471', '앱스토어', '단', '9964837032', '1500', '24', '혜택', '8093', '478-85', '할부', 'NO', '지다', '성', ',', '빅맨', '불가

In [216]:
intersection = set(s1).intersection(set(s2))
print(intersection)

{'인천', '30일', '점', '롯데', '차다', '★', '승인', '세', '시', '하다', '919490', '앱', '과세', ':', '04', '22255237', '담당자', '사명', '대표자', '환불', '금액', '정부', '00', '가다', '합', '이내', '280', '남다', '휴게', '가액', '을', '변심', '에서', '-', '부', '8801121025130', '.', '단가', '광동', 'e', '(', '월', '요구르트', '가맹', '검색', '상품', '번호', '서구', '명', '으로', 'POS', '8806002011967', ')', '사업자', '시불', '이마트', '245', '성열', '에', '의하다', '02', '150', '기다', '가능하다', '가져가다', '받다', '/', '단순', '영수증', '수량', '카드', '25', '"', '식품', '보다', '빠르다', '결제', '33', '03145', '신용카드', '01', '영종대교', '앱스토어', '단', '9964837032', '24', '8093', '할부', 'NO', '지다', ',', '빅맨', '불가', '0001', '물품', '방침', '개', '쉬다', '를', '교환', '법인', '일', '는', '문의', '2,700', '정서진', '00613', '혜택', '1', '[', 'ml'}


In [217]:
print(len(intersection)/len(union))

0.6981132075471698


###영수증 유사도 검출(형태소 분리X)

In [177]:
f = open('/content/drive/My Drive/문서 유사도/24659469.txt', 'r')
#f = open('/content/drive/My Drive/문서 유사도/24659469_masked.txt', 'r')
f2 = open('/content/drive/My Drive/문서 유사도/24659469_hand.txt', 'r')
user = f.read()
ground = f2.read()
print(user)

rio emart24
이마트24 영종대교 게소점
대표자 : 김윤아, 성열 기
사업자번호 : 478 85 -00613
인천시 서구 정서진남로 25, 영종대교 휴게
103145 2020 - 04 - 02 13 : 09 POS -0001 471
상품명 단가 수량 금액
01) 광동 ) 대추상화 150m /
8806002011967 1, 500 1 1, 500
02) e) 빅맨요구르트 280ml
8801121025130 1, 200 1 1, 200
과세물품가액 2 ,455
부 가 세 245
합받 계액 2 2, ,700 700
을
신용카드 2,100
드
카드번호 : 4670-09**-** 來案- 8093 1
카드사명: 롯데법인카드
할부개월: 00 일시불
결제금액: 2 , 700
승인번호 : 04 22255237
가맹점번호 : 9964837032
NO :919490 담당자 :03145 [김운아, 성열 ]
★ 가맹문의 : 02 -6916-1500
정부방침에 의해 교환/ 환불시 영수증을
지참하셔야 하며 , 카드결제는 30일 이내
카드/ 영수증 지참시 가능합니다.
( 단 , 식품 단순변심 교환/환불 불가)
★이마트24 앱으로 보다 쉽고 빠른
혜택을 가져가세요.
앱스토어에서 "이마트24" 를 검색하세요 ★
10202004020314500019194901


In [0]:
tokenized_user = user.split()
tokenized_ground = ground.split()

In [179]:
union = set(tokenized_user).union(set(tokenized_ground))
print(union)

{'대추상화', ',455', '85', '의해', '30일', '478-85-00613', '13', '★', '상품명', '세', '하며,', '가', '2020-04-02', '검색하세요★', '1,200', '가능합니다.', ':', '카드/영수증', '04', '2,100', '150m', '280ml', '단순변심', '22255237', '02-6916-1500', '담당자', ':03145', '대표자', '정서진남로', '금액', '롯데법인카드', ':919490', '쉽고', '불가)', '00', '합', '이내', '드', '승인번호:', 'NO:919490', '휴게', '09', '교환/환불시', '하며', '혜택을', '을', '"이마트24"를', '-', '부', '500', '환불시', '8801121025130', '단가', '광동', '(', '영수증을', '교환/', '---------------------------------------------------', '담당자:03145[김운아,성열*]', '478', '김운아,성열기', '2,455', '앱스토어에서', '700', '1,500', '서구', 'emart24', '할부개월:', 'POS', '가맹점번호:', '8806002011967', ')', '결제금액:', '승인번호', '13:09', '광동)대추쌍화150ml', '카드번호:', '지참하셔야', '성열', '245', '4670-09**-****-8093', '인천시', '02)', '02', '지참시', '빅맨요구르트', '/', '가맹문의', '계액', '영수증', '10202004020314500019194901', '[김운아,', '-00613', '김윤아,', '과세물품가액', '수량', '가맹점번호', '020200402031450001919490', '식품', '보다', 'POS-0001', '2020', '103145', '33', '200', '03145', '계', '앱으로', '신용카드

In [181]:
intersection = set(tokenized_user).intersection(set(tokenized_ground))
print(intersection)

{'휴게', '9964837032', '혜택을', '의해', '30일', '부', '가맹문의', '카드결제는', '★', '일시불', '8801121025130', '상품명', '세', '단가', '사업자번호', '가', '과세물품가액', '01)', '영수증을', '교환/환불', '수량', '카드사명:', '보다', '앱스토어에서', '가능합니다.', ':', '서구', 'emart24', '04', '단순변심', '할부개월:', '22255237', '8806002011967', '대표자', '가져가세요.', '결제금액:', '정서진남로', '신용카드', '금액', '롯데법인카드', '지참하셔야', '245', '불가)', '00', '인천시', '02)', '영종대교', '25,', '이마트24', '이내', '정부방침에', '1', '지참시'}


In [182]:
print(len(intersection)/len(union))

0.33974358974358976
